In [1]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)

from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule 
import pandas as pd
from scdataloader.utils import load_genes

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Global seed set to 42


💡 connected lamindb: jkobject/scprint


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/chex/_src/pytypes.py:53: DeprecationWarning: jax.core.Shape is deprecated. Use Shape = Sequence[int | Any].
  Shape = jax.core.Shape
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/chex/_src/pytypes.py:54: DeprecationWarning: jax.random.KeyArray is deprecated. Use jax.Array for annotations, and jax.dtypes.issubdtype(arr.dtype, jax.dtypes.prng_key) for runtime detection of typed prng keys (i.e. keys created with jax.random.key).
For more information, see https://jax.readthedocs.io/en/latest/jep/9263-typed-keys.html
  PRNGKey = jax.random.KeyArray
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/scib_metrics/_types.py:9: DeprecationWarning: jax.random.KeyArray is deprecated. Use jax.Array for annotations, and jax.dtypes.i

In [2]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_clss = [
    "cell_type_ontology_term_id",  # 1
    # "tissue_ontology_term_id",
    "disease_ontology_term_id",  # 2
    #    "development_stage_ontology_term_id",
    "assay_ontology_term_id",  # 3
    'self_reported_ethnicity_ontology_term_id',  # 4
]
clss_to_pred = hierarchical_clss+[
    'sex_ontology_term_id',  # 5
    "organism_ontology_term_id",  # 6
]
all_clss = clss_to_pred+[
    # 'dataset_id',
    # 'cell_culture',
    #  "heat_diff",
    #  "total_counts",
    # "nnz",
    #  "dpt_group",
]

gene_emb = '../data/main/gene_embeddings.parquet'
d_model = 128

In [3]:
datamodule = DataModule(
    collection_name="some", #some, all, preprocessed dataset, all no zhang, 
    gene_embeddings=gene_emb,
    all_clss=all_clss,
    hierarchical_clss=hierarchical_clss,
    organisms=["NCBITaxon:9606"],  # , "NCBITaxon:10090"],
    how="most expr",
    max_len=1200,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category
    weight_scaler=10,
    clss_to_weight=clss_to_pred,
    clss_to_pred=clss_to_pred,
    batch_size=1,
    num_workers=1,
    # train_oversampling=2,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    test_split=0.05)
testfiles = datamodule.setup()

won't do any check but we recommend to have your dataset coming from local storage

80.0% are aligned
seeing a string: loading gene positions as biomart parquet file


In [4]:
# create a function to transform an scGPT checkpoint to an scPrint's
# ckpt = torch.load("../../scGPT/save/model_e6.pt")
# scPrint.load_from_checkpoint("../../scGPT/save/model_e6.pt")

In [5]:
# from lightning.pytorch.profilers import PyTorchProfiler
# pytorch_prof = PyTorchProfiler("../data/tensorboard", emit_nvtx=False, group_by_input_shape=True, record_shapes=True, profile_memory=True, with_stack=True, on_trace_ready=torch.profiler.tensorboard_trace_handler("../data/tensorboard/"),)

In [13]:
model.save_pretrained("test")

In [10]:
scPrint.push_to_hub("jkobject/scPRINT")

pytorch_model.bin:   0%|          | 0.00/120M [00:00<?, ?B/s]

TypeError: scPrint.__init__() missing 1 required positional argument: 'genes'

In [7]:
model.device.type

'cuda'

In [5]:
model = model.to("cuda")
with torch.autocast(device_type="cuda", dtype=torch.float16):
    for batch in datamodule.train_dataloader():
        gene_pos, expression, depth = (batch["genes"], batch["x"], batch["depth"])
        gene_pos, expression, depth = gene_pos.to("cuda"), expression.to("cuda"), depth.to("cuda")
        res = model._predict(gene_pos, expression, depth)
        break


In [4]:
model = scPrint(
    genes=datamodule.genes,
    d_model=d_model*4,
    nhead=4*2,
    nlayers=1,
    layers_cls = [d_model],
    classes = datamodule.classes,
    labels_hierarchy = datamodule.labels_hierarchy,
    dropout=0,
    transformer="flash",
    precpt_gene_emb=gene_emb,
    gene_pos_enc=datamodule.gene_pos,
    mvc_decoder="inner product",
    label_decoders = datamodule.decoders,
    fused_dropout_add_ln=False,
    num_batch_labels = datamodule.num_datasets,
    checkpointing=False,
    prenorm=True,
    num_heads_kv=None,
    #weight_decay=0.01,
    #zinb=False
)

In [6]:
# from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="scprint_test",
                           save_dir="../data/tensorboard")
wandb_logger.watch(model, log='all', log_freq=50, log_graph=True)

# tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
# tlogger.log_graph(model)

2024-05-22 13:05:20,668:ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jkobject (ml4ig). Use `wandb login --relogin` to force relogin
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [1]:
import gseapy as gp

In [2]:
tfchip = gp.get_library(name="ENCODE_TF_ChIP-seq_2014")


In [4]:
import json

# Save the dictionary to a JSON file
with open('../data/tfchip_data.json', 'w') as file:
    json.dump(tfchip, file)

# Load the dictionary back from the JSON file
with open('tfchip_data.json', 'r') as file:
    tfchip_loaded = json.load(file)


In [7]:
chckp = ModelCheckpoint(monitor="val_loss", save_top_k=-1)
trainingmode = TrainingMode(
    do_denoise=True,
    noise=[0.01],
    do_cce=False,
    cce_sim=0.6,
    do_ecs=False,
    ecs_threshold=0.4,
    ecs_scale=0.05,
    class_scale=0.08,
    do_cls=False,
    do_mvc=False,
    do_adv_cls=False,
    do_next_tp=False,
    mask_ratio=[],
    warmup_duration=100,
    fused_adam=True,
    lr_reduce_patience=200,
)
# es = EarlyStopping(patience=2, monitor='val_loss')
# swa = StochasticWeightAveraging(swa_lrs= 0.01)
# lrm = LearningRateMonitor(logging_interval="step")
# lrf = LearningRateFinder(mode="exponential",)
# TODO: to check that the class hierarchy are really ordered from 1-2-3-4... as well (oredered dict)
# , logger=tlogger) #detect_anomaly=True, fast_dev_run=20, overfit_batches=10, limit_train_batches=1, limit_val_batches=0
trainer = Trainer(precision="16-mixed", gradient_clip_val=500, max_time={"hours": 2}, limit_val_batches=1, callbacks=[
                  trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=1, reload_dataloaders_every_n_epochs=1000000, logger=wandb_logger)
# logger=wandb_logger,

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


In [7]:
# sanity. should be overfiting.
trainer = Trainer(precision="16-mixed", gradient_clip_val=500, max_time={"hours": 2}, limit_val_batches=1, callbacks=[
                  trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=1, overfit_batches=1, reload_dataloaders_every_n_epochs=1000_000, logger=wandb_logger, num_sanity_val_steps=0)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [ ]:
from scprint.tasks.grn import default_benchmark

In [9]:
model.lr_reduce_patience = 50
model.lr_reduce_ratio = 0.2

In [10]:
model.lr = 0.00001

In [ ]:
default_benchmark(model, default_dataset="../data/yBCKp6HmXuHa0cZptMo7.h5ad") #gwps, other

/home/ml4ig1/Documents code/scPRINT/scprint/tasks/grn.py:515: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  organisms=adata.obs['organism_ontology_term_id'][0],
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ml4ig1/Documents code/scPRINT/scprint/tasks/grn.py:167: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  organisms=[subadata.obs['organism_ontology_term_id'][0]],
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

avg link count: 536246649, sparsity: 1.0
base enrichment
too many genes for central computation
_________________________________________
TF specific enrichment


2024-05-22 13:16:10,303:INFO - Downloading and generating Enrichr library gene sets...
2024-05-22 13:16:10,305:INFO - Library is already downloaded in: /home/ml4ig1/.cache/gseapy/Enrichr.ENCODE_TF_ChIP-seq_2014.gmt, use local file
2024-05-22 13:16:10,530:INFO - 0332 gene_sets have been filtered out when max_size=2000 and min_size=0


found some significant results for  7.142857142857143 % TFs

_________________________________________
loading GT,  omnipath


...

/home/ml4ig1/Documents code/benGRN/bengrn/base.py:277: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if i in genes and j in genes:


true elem 14408 ...
doing regression....


/home/ml4ig1/Documents code/benGRN/bengrn/base.py:303: RuntimeWarning: invalid value encountered in long_scalars
  "recall": (pred[y_test == 1] == 1).sum() / y_test.sum(),


ValueError: too many values to unpack (expected 2)

: 

In [9]:
trainer.fit(model, datamodule=datamodule)

In [ ]:
weirdly, when input expression is set to 0 and pi is set to 1000. I get a loss of 0
when input expression is set to some set of values and pi=-10, disp=max, mean=input, I get 0.479, decreasing pi doesn't help, making it higher, makes it worse, max of disp is ~3M afterward it increases back (likely due to some overflow??). I am guessing that it would need to be set to infinity for it to work. might be better to use dispersion instead of inverse dispersion?

all things equal, if disp is set to 2_700_000, I get loss tensor(3.8694, device='cuda:0')
if disp is 2_700_000+1 -> tensor(0.1994, device='cuda:0')

basically it seems that at too high values or at too low values we get strong instabilities that are exacerbated by the fact that we are in float16.
could it also be because of the espilon? nope

-> why doesn't my model go to this low loss value??



In [ ]:
1. there seems to be an in-between value of disp that makes sense given the expression data
2. 

In [ ]:
import torch.nn.functional as F

In [ ]:
F.mse_loss(expression, expression, reduction="mean")

In [ ]:
loss.nb(
    theta=disp,
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"],
    pi=output["zero_logits"],
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"] + 10000,
    pi=output["zero_logits"]-3,
    mu=expression,
    target=expression,
)

In [ ]:
loss.nb(
    theta=output["disp"],
    mu=output["mean"],
    target=expression,
).mean()

In [ ]:
expr = expression.clone()
expr[:,-20:] = 0
expr[:,:20] +=1

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=1e4*output["mean"]/expression.sum(),
    target=1e4*expression/expression.sum(),
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=1e4*expr/expression.sum(),
    target=1e4*expression/expression.sum(),
)

In [ ]:
exp(15)

In [ ]:
disp = torch.zeros_like(output["disp"])
pi = torch.zeros_like(output["disp"]) - 10

In [ ]:
# why if it is the inverse dispersion? does setting it to 3_000_000 works but not 10_000_000?
# maybe some overflow??

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"],
    pi=output["zero_logits"],
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=expression,
    target=expression,
)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="../data/tensorboard"

In [ ]:
# wandb_logger.finalize(status="aborted")
torch.cuda.empty_cache()